## Part 2: Data Cleaning

### Purpose
This notebook cleans and transforms the raw YouTube trending videos dataset  
collected in **01_data_collection.ipynb** to prepare it for analysis.

---

### Steps
1. Load the raw dataset.
2. Add a `video_url` column from `video_id`.
3. Convert `published_at` to datetime and extract time-based features.
4. Ensure numerical columns are stored as integers.
5. Clean and standardize text fields.
6. Remove duplicates if necessary.
7. Save the cleaned dataset to `data/processed/`.

### Input
- `data/raw/trending_videos_ke.csv`

### Output
- `data/processed/trending_videos_ke_clean.csv`


#### Step 1. Import All Required libraries

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
import os

#### Load the raw dataset

In [2]:
df= pd.read_csv('data/raw/trending_videos_ke.csv')

df.head(5)

,video_id,title,description,published_at,channel_id,channel_title,category_id,category_name,tags,duration,definition,caption,view_count,like_count,favorite_count,comment_count
0,R9f4GST2yQQ,Zuchu - Amanda (Visualizer),Get #Amanda Now 👇🏼\nhttps://zuchu.lnk.to/amand...,2025-08-01T09:57:57Z,UCOn8zDM533kqzhIA8c3NFbQ,Zuchu,10,Music,"['diamond platnumz', 'wana', 'kwaru', 'hakuna ...",PT2M39S,hd,False,880369,22857,0,2280
1,nb_fFj_0rq8,Avatar: Fire and Ash | Official Trailer,"From Director James Cameron, watch the trailer...",2025-07-28T15:00:09Z,UCgjxQJ6TlKqhHax8742ZMdA,Avatar,24,Entertainment,[],PT2M26S,hd,True,19113385,507686,0,28335
2,nYH0vXWZ69k,Live : Kenya vs D.R. Congo | African Nations C...,Live : Kenya vs D.R. Congo | African Nations C...,2025-08-03T16:18:41Z,UC4P_ujgWcdjfNW2z5eQVMyg,Asm Power Gamer,20,Gaming,"['kenya vs dr congo live football', 'kenya dr ...",PT3H23M15S,hd,False,309576,714,0,2
3,qZgEdZFzkVc,MATATA ft. MEJJA - TIKI TAKO [Official Music V...,Matata ft. Mejja – Tikitako (Official Music Vi...,2025-07-25T05:23:48Z,UCJ2XnwRKT0ksT9E1lri1wlg,MATATA OFFICIAL,10,Music,"['Matata ft mejja', 'matata tikitako', 'tiki t...",PT4M2S,hd,False,1035173,21736,0,1189
4,5r-7eWDBc40,GOAT - Official Teaser Trailer (HD),You’re never too small to dream big.\n\nFrom t...,2025-07-30T12:59:45Z,UCz97F7dMxBNOfGYu3rx8aCw,Sony Pictures Entertainment,24,Entertainment,[],PT2M14S,hd,True,27193369,110500,0,7541


#### Step 3. Cleaning & Transformation

**Part A:** Generate `video_url` from `video_id`

In [3]:
# Generate full video url using the video_id
df['video_url']= "https://www.youtube.com/watch?v=" + df['video_id']

df[['video_id','video_url']].head(3)

,video_id,video_url
0,R9f4GST2yQQ,https://www.youtube.com/watch?v=R9f4GST2yQQ
1,nb_fFj_0rq8,https://www.youtube.com/watch?v=nb_fFj_0rq8
2,nYH0vXWZ69k,https://www.youtube.com/watch?v=nYH0vXWZ69k


**Part B:** Convert `published_at` to datetime and extract time-based features

In [4]:
# Convert published_at to datetime
df['published_at'] = pd.to_datetime(df['published_at'], errors='coerce')

In [5]:
# Extract date only
df['publish_date'] = df['published_at'].dt.date

In [6]:
# Get time only
df['publish_time'] = df['published_at'].dt.time

In [7]:
# Get day of week (Monday, Tuesday, etc.)
df['day_of_week'] = df['published_at'].dt.day_name()

In [8]:
# Get hour (0-23)
df['hour'] = df['published_at'].dt.hour

In [9]:
# Changes made
df[['published_at', 'publish_date', 'publish_time', 'day_of_week', 'hour']].head()

,published_at,publish_date,publish_time,day_of_week,hour
0,2025-08-01 09:57:57+00:00,2025-08-01,09:57:57,Friday,9
1,2025-07-28 15:00:09+00:00,2025-07-28,15:00:09,Monday,15
2,2025-08-03 16:18:41+00:00,2025-08-03,16:18:41,Sunday,16
3,2025-07-25 05:23:48+00:00,2025-07-25,05:23:48,Friday,5
4,2025-07-30 12:59:45+00:00,2025-07-30,12:59:45,Wednesday,12


**Part C:** Convert numeric columns to integers


In [10]:
# List of numeric columns to be  converted
numeric_cols = ['view_count', 'like_count', 'favorite_count', 'comment_count']


In [11]:
for col in numeric_cols:
    df[col]=pd.to_numeric(df[col], errors='coerce')

df[numeric_cols].head(5)

,view_count,like_count,favorite_count,comment_count
0,880369,22857,0,2280
1,19113385,507686,0,28335
2,309576,714,0,2
3,1035173,21736,0,1189
4,27193369,110500,0,7541


**Part D:** Convert `duration` from ISO 8601 format to total seconds and readable time

In [18]:
pip install isodate

  Using cached isodate-0.7.2-py3-none-any.whl.metadata (11 kB)
Using cached isodate-0.7.2-py3-none-any.whl (22 kB)
Note: you may need to restart the kernel to use updated packages.


In [12]:
import isodate

In [13]:
def convert_duration_to_seconds(duration_str):
    """
    Converts YouTube API ISO 8601 duration (e.g., 'PT2M39S') to total seconds.
    Returns None if input is NaN or invalid.
    """
    if pd.isnull(duration_str):
        return None
    try:
        return int(isodate.parse_duration(duration_str).total_seconds())
    except:
        return None

In [14]:
# Apply conversion to seconds
df['duration_seconds'] = df['duration'].apply(convert_duration_to_seconds)

# Convert seconds to minutes (rounded to 2 decimal places)
df['duration_minutes'] = df['duration_seconds'] / 60

# Preview
df[['duration', 'duration_seconds', 'duration_minutes']].head()

,duration,duration_seconds,duration_minutes
0,PT2M39S,159,2.650000
1,PT2M26S,146,2.433333
2,PT3H23M15S,12195,203.250000
3,PT4M2S,242,4.033333
4,PT2M14S,134,2.233333


**Part E(i):** Clean text encoding issues in `description`

In [15]:
# Function to fix bad encoding in text
def clean_text_encoding(text):
    """
    Fixes common text encoding issues (UTF-8 misinterpretation)
    and removes excessive whitespace.
    """
    if pd.isnull(text):
        return text
    try:
        # Fix misencoded UTF-8
        fixed_text = text.encode('latin1').decode('utf-8')
    except:
        fixed_text = text  # If decoding fails, keep the original text

    # Remove excessive spaces and newlines
    fixed_text = " ".join(fixed_text.split())

    return fixed_text

In [16]:
# Apply the function to description column
df['description'] = df['description'].apply(clean_text_encoding)

In [17]:
df[['description']].head(5)

,description
0,Get #Amanda Now 👇🏼 https://zuchu.lnk.to/amanda...
1,"From Director James Cameron, watch the trailer..."
2,Live : Kenya vs D.R. Congo | African Nations C...
3,Matata ft. Mejja – Tikitako (Official Music Vi...
4,You’re never too small to dream big. From the ...


**Part E(ii):** Remove trailing/leading spaces in `title` and `tags`

In [18]:
df['title'] = df['title'].apply(clean_text_encoding)

In [19]:
# --- Function to strip spaces from tags ---
def strip_spaces_from_tags(tags):
    """
    Strips leading/trailing spaces from each tag if tags is a list.
    """
    if isinstance(tags, list):
        return [tag.strip() for tag in tags]
    return tags

# Apply to tags column
df['tags'] = df['tags'].apply(strip_spaces_from_tags)


In [20]:
df[['title', 'tags']].head(3)

,title,tags
0,Zuchu - Amanda (Visualizer),"['diamond platnumz', 'wana', 'kwaru', 'hakuna ..."
1,Avatar: Fire and Ash | Official Trailer,[]
2,Live : Kenya vs D.R. Congo | African Nations C...,"['kenya vs dr congo live football', 'kenya dr ..."


In [21]:
df.isnull().sum()

video_id            0
title               0
description         9
published_at        0
channel_id          0
channel_title       0
category_id         0
category_name       0
tags                0
duration            0
definition          0
caption             0
view_count          0
like_count          0
favorite_count      0
comment_count       0
video_url           0
publish_date        0
publish_time        0
day_of_week         0
hour                0
duration_seconds    0
duration_minutes    0
dtype: int64

#### Step 4. Save cleaned dataset

In [22]:
df.columns

Index(['video_id', 'title', 'description', 'published_at', 'channel_id',
       'channel_title', 'category_id', 'category_name', 'tags', 'duration',
       'definition', 'caption', 'view_count', 'like_count', 'favorite_count',
       'comment_count', 'video_url', 'publish_date', 'publish_time',
       'day_of_week', 'hour', 'duration_seconds', 'duration_minutes'],
      dtype='object')

In [23]:
# Reorder columns
column_order = [
    'video_id', 'video_url' , 'title', 'description', 'published_at', 'publish_date', 'publish_time',
    'day_of_week', 'hour','channel_id',
    'channel_title', 'category_id', 'category_name', 'tags', 'duration','duration_seconds', 'duration_minutes',
    'definition', 'caption', 'view_count', 'like_count', 'favorite_count',
    'comment_count'
]


In [24]:
df = df[column_order]

In [25]:
# Define output path
output_path = 'data/processed/trending_videos_ke_clean.csv'

# Create folder if it doesn't exist
os.makedirs(os.path.dirname(output_path), exist_ok=True)

In [26]:
# Save CSV
df.to_csv(output_path, index=False)
print(f"Cleaned dataset saved to {output_path}")

Cleaned dataset saved to data/processed/trending_videos_ke_clean.csv
